In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib notebook

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [24]:
#Evalutaion metric
def rmsle(y_pred, y_test) : 
    assert len(y_test) == len(y_pred)
    return np.sqrt(np.mean((np.log(1+y_pred) - np.log(1+y_test))**2))

In [3]:
train.head()

,id,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree,formation_energy_ev_natom,bandgap_energy_ev
0,1,33,80.0,0.6250,0.3750,0.000,9.9523,8.5513,9.1775,90.0026,90.0023,90.0017,0.0680,3.4387
1,2,194,80.0,0.6250,0.3750,0.000,6.1840,6.1838,23.6287,90.0186,89.9980,120.0025,0.2490,2.9210
2,3,227,40.0,0.8125,0.1875,0.000,9.7510,5.6595,13.9630,90.9688,91.1228,30.5185,0.1821,2.7438
3,4,167,30.0,0.7500,0.0000,0.250,5.0036,5.0034,13.5318,89.9888,90.0119,120.0017,0.2172,3.3492
4,5,194,80.0,0.0000,0.6250,0.375,6.6614,6.6612,24.5813,89.9960,90.0006,119.9893,0.0505,1.3793


In [4]:
# convert space_groups and number_of_total_atoms to one hot encoder
space_groups=pd.get_dummies(train.spacegroup)
num_atoms=pd.get_dummies(train.number_of_total_atoms)


In [11]:
train_one=pd.concat([train, space_groups,num_atoms ],axis=1)
train_one=train_one.set_index('id') # set id as index


In [12]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split( train_x, train_y, test_size=0.20, random_state=42)

In [19]:
#fitting y values
train_y=train_one.loc[:,['formation_energy_ev_natom','bandgap_energy_ev']]
#fitting x values so we need to remove y values as well as spacegroup and number_of _total_atoms column
train_x=train_one.drop(['formation_energy_ev_natom','bandgap_energy_ev','spacegroup','number_of_total_atoms'],axis=1)

In [20]:
from xgboost.sklearn import XGBRegressor

In [69]:
xgr=XGBRegressor(nthread=-1)
xgr_Eg=XGBRegressor(nthread=-1)

In [70]:
y_train_log=np.log(y_train['formation_energy_ev_natom']+1)
y_test_log=np.log(y_test['formation_energy_ev_natom']+1)

In [55]:
xgr.fit(X=X_train,y=y_train_log,eval_metric='rmse')

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=-1, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [60]:
print("Train score:")
print(xgr.score(X_train,y_train_log))
print("Test score:")
print(xgr.score(X_test,y_test_log))

Train score:
0.8990562496997951
Test score:
0.8166374079649013


In [68]:
print("RMSLE train error:")
print(rmsle(np.exp(xgr.predict(X_train))-1,y_train.formation_energy_ev_natom))
print("RMSLE test error:")
print(rmsle(np.exp(xgr.predict(X_test))-1,y_test.formation_energy_ev_natom))

RMSLE train error:
0.02759030299630954
RMSLE test error:
0.0365499675179003


In [67]:
np.exp(xgr.predict(X_train)-1).shape

(1920,)